In [1]:
import os
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

<strong>There are 2 datasets:</strong><br/>
<ol>
<li>Training data set,</li>
<li>Test data set.</li>
</ol>
Each of these data sets are .csv files. <br/>
Each dataset has 785 columns with a variable number of rows.
We shall use the pandas package to read these files. The results are numpy arrays.

In [2]:
training_filename="C:\Apps\Projects\ml\data\mnist\mnist_train.csv"
tmp=pd.read_csv(training_filename, sep=',', header=None)
# train_nrows, train_ncols=tmp.shape
x_train, y_train=tmp.loc[:,1:tmp.shape[1]].values, tmp.loc[:,0].values
test_filename="C:\Apps\Projects\ml\data\mnist\mnist_test.csv"
tmp=pd.read_csv(test_filename, sep=',', header=None)
#test_nrows, test_ncols=tmp.shape
x_test, y_test=tmp.loc[:,1:tmp.shape[1]].values, tmp.loc[:,0].values
tmp=[]

In [6]:
logs_path = 'logging/' # logging path
batch_size = 100 # batch size while performing training 
learning_rate = 0.003 # Learning rate 
training_epochs = 20 # training epoch
display_epoch = 1
layer_count = np.array([300, 300, 100, 60, 10])

In [7]:
X = tf.placeholder(tf.float32, shape=(None, 784), name='X_Vector')
XX = tf.reshape(X, [-1, 784])
Y_  = tf.placeholder(tf.float32, shape=(None, 10), name='Y_Vector')
lr = tf.placeholder(tf.float32)

In [8]:
# Create the weights and bias vector
W = []
B = []
Y = []
n = len(layer_count)
assert layer_count[-1] == 10, "The final layer must have 10 nodes."
for i in range(n) :
    #print(i)
    B.append(tf.Variable(tf.zeros([layer_count[i]])))
    if i == 0 :
        W.append(tf.Variable(tf.truncated_normal([784, layer_count[i]], stddev=0.1)))
        Y.append(tf.nn.sigmoid(tf.matmul(XX, W[i]) + B[i]))
    else :
        W.append(tf.Variable(tf.truncated_normal([layer_count[i-1], layer_count[i]], stddev=0.1)))
        if (i < (n-1)) :
            Y.append(tf.nn.sigmoid(tf.matmul(Y[i-1], W[i]) + B[i]))            
Ylogits = tf.matmul(Y[n-2], W[n-1]) + B[n-1]
Y.append(tf.nn.softmax(Ylogits))
    #print(layer_count[i])
#W = tf.Variable(tf.truncated_normal([784, N1], stddev=0.1))
# Create the bias term
#B1 = tf.Variable(tf.zeros([N1])) 

In [9]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
cost_op = tf.reduce_mean(cross_entropy)*100

In [10]:
correct_prediction = tf.equal(tf.argmax(Y[-1], 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
# Optimization op (backprop)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost_op)

In [12]:
# Create a summary to monitor cost tensor
tf.summary.scalar("cost", cost_op)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
summary_op = tf.summary.merge_all()

In [13]:
init_op = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init_op)
    avg_cost = 0.
    # op to write logs to TensorBoard
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    yM_train = np.zeros(shape=(len(y_train), 10), dtype=float)
    for i in range(len(y_train)):
        yM_train[i, y_train[i]] = 1
    yM_test = np.zeros(shape=(len(y_test), 10), dtype=float)
    for i in range(len(y_test)):
        yM_test[i, y_test[i]] = 1
    for epoch in range(training_epochs):
        batch_count = int(len(y_train)/batch_size)
        for i in range(batch_count):
            start_row = i*batch_size
            end_row_plus_one = (i+1)*batch_size
            # Change this
            max_learning_rate = 0.003
            min_learning_rate = 0.0001
            decay_speed = 2000 
            batch_x, batch_y = x_train[start_row:end_row_plus_one, :], yM_train[start_row:end_row_plus_one,:]

            learning_rate = min_learning_rate+\
                            (max_learning_rate - min_learning_rate)\
                            * math.exp(-i/decay_speed)
            _, c,summary = sess.run([train_op, cost_op, summary_op], feed_dict={X: batch_x, Y_: batch_y, lr: learning_rate})
            avg_cost += c / batch_count
            writer.add_summary(summary, epoch * batch_count + i)
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%2d' % (epoch+1), "cost=", "{:.2f}".format(avg_cost))   
        print("Epoch: ", epoch)
    print("Optimization Finished!")
           
    print("Accuracy: ", accuracy.eval(feed_dict={X: x_test, Y_: yM_test}))

Epoch:  1 cost= 71.22
Epoch:  0
Epoch:  2 cost= 112.88
Epoch:  1
Epoch:  3 cost= 148.59
Epoch:  2
Epoch:  4 cost= 182.63
Epoch:  3
Epoch:  5 cost= 212.50
Epoch:  4
Epoch:  6 cost= 240.76
Epoch:  5
Epoch:  7 cost= 268.16
Epoch:  6
Epoch:  8 cost= 293.97
Epoch:  7
Epoch:  9 cost= 319.63
Epoch:  8
Epoch: 10 cost= 345.50
Epoch:  9
Epoch: 11 cost= 370.20
Epoch:  10
Epoch: 12 cost= 393.26
Epoch:  11
Epoch: 13 cost= 417.03
Epoch:  12
Epoch: 14 cost= 438.48
Epoch:  13
Epoch: 15 cost= 460.30
Epoch:  14
Epoch: 16 cost= 481.24
Epoch:  15
Epoch: 17 cost= 501.44
Epoch:  16
Epoch: 18 cost= 521.51
Epoch:  17
Epoch: 19 cost= 541.98
Epoch:  18
Epoch: 20 cost= 560.90
Epoch:  19
Optimization Finished!
Accuracy:  0.9306


In [15]:
index=5
print(yM_test[index,:])
print(y_test[index])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1
